In [ ]:
import io
import random
import string # to process standard Python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
# Authenticate and create the API client
auth.authenticate_user()
docs_service = build('docs', 'v1')

In [ ]:
pip install nltk #Natural Language Toolkit

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

In [ ]:
DOCUMENT_ID = '1GnUUSg_0qENzcgj85gJQa6Ou18zfd0xM_deqYZultWo'

def get_document_text(doc_id):
    try:
        doc = docs_service.documents().get(documentId=doc_id).execute()
        content = doc.get('body').get('content')

        text = ''
        for element in content:
            if 'paragraph' in element:
                elements = element['paragraph']['elements']
                for elem in elements:
                    if 'textRun' in elem:
                        text += elem['textRun']['content']
        return text
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

# Fetch and process the document's content
raw = get_document_text(DOCUMENT_ID)
if raw:
    raw = raw.lower()  # Convert to lowercase
    print(raw)


coca-cola statement of work
 project objectives 
the main objective of this project (1) is to train and develop a chatbot that summarizes, diagnoses and identifies dispenser issues and assist coca-cola teams in querying and understanding data related to dispenser performance and maintenance. this will also help with the following:
·   	increase engineering productivity associated with java log analysis
·   	proactive identification and preventative maintenance from developed heuristics and algorithms
 
business questions
the following are some of the business questions that we will answer:
·   	can we track the total number of times the door has been opened for each freestyle machine, and what patterns can we identify related to machine usage or potential maintenance events?
how can we use the total number of reboots per machine to identify potential maintenance issues or early signs of malfunctioning equipment?
·   	can the user-friendly prompts in chatbot help coca-cola teams identif

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [ ]:
flag=True
print("My name is Freestyle Genie. I will answer your queries about Coca-Cola. To exit, type cola!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='cola'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Freestyle Genie:"+greeting(user_response))
            else:
                print("Freestyle Genie:",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Freestyle Genie: Bye! take care..")

My name is Freestyle Genie. I will answer your queries about Coca-Cola. To exit, type cola!
hi
Freestyle Genie:hey
what is door open activity
Freestyle Genie:the chatbot will provide insights on the following key activities:
door open activity: track the total number of door open activities per machine, with analysis to identify patterns that could indicate potential maintenance or improper usage.
tell me about reboot analysis
Freestyle Genie:·   	can the user-friendly prompts in chatbot help coca-cola teams identify potential maintenance needs by analyzing door open events, pour activity, reboot summaries, trouble analysis, and comparative analysis to prevent machine failures?
cola
Freestyle Genie: Bye! take care..
